In [2]:
import pandas as pd
import numpy as np
from random import gauss




In [6]:
# Load the individual CSV files for each district
district_files = ["Bellary_inflation_adjusted_prices.csv", "Haveri_inflation_adjusted_prices.csv", "Gadag_inflation_adjusted_prices.csv", "Chitradurga_inflation_adjusted_prices.csv", "Davanagere_inflation_adjusted_prices.csv"]  # Replace with the file paths of the individual CSV files

crop_prices = {}  # Dictionary to store crop prices for each district

In [14]:
crop_prices = {}  # Dictionary to store crop prices for each district
districts = []  # List to store district names
for file in district_files:
    district = pd.read_csv(file)["Dist Name"].iloc[0]  # Extract the district name from the file
    districts.append(district)  # Add district name to the list
    crop_prices[district] = pd.read_csv(file, usecols=["Year", "PADDY", "SORGHUM", "MAIZE", "FINGER MILLET", "CHICKPEA", "PIGEONPEA", "GROUNDNUT"]).set_index('Year')  # Store the crop prices with the district as the key



In [39]:
# Step 2: Calculate the mean and volatility of the price for each crop
crop_stats = {}  # Dictionary to store crop statistics (mean and volatility)
for crop in crop_prices[districts[0]].columns:
    crop_stats[crop] = {
        'Mean': np.mean([crop_prices[district][crop].mean() for district in districts]),
        'Volatility': np.mean([crop_prices[district][crop].std() for district in districts])
    }

print(crop_stats)

{'PADDY': {'Mean': 986.2842656706592, 'Volatility': 167.68486870916567}, 'SORGHUM': {'Mean': 983.1492510450491, 'Volatility': 297.91078041453505}, 'MAIZE': {'Mean': 885.9443729606324, 'Volatility': 159.1065629374519}, 'FINGER MILLET': {'Mean': 844.9301580351482, 'Volatility': 226.6319391284873}, 'CHICKPEA': {'Mean': 2384.4277783362186, 'Volatility': 576.9657512764799}, 'PIGEONPEA': {'Mean': 3035.6584485295466, 'Volatility': 1078.4316596535396}, 'GROUNDNUT': {'Mean': 2612.9154163776484, 'Volatility': 565.4292776762829}}


In [49]:

# Build a random walk model to predict prices for the next 25 years
num_years = 25
prediction_years = range(2017, 2017 + num_years)
predicted_prices = {}  # Dictionary to store predicted prices for each district
for district in crop_prices:
    predicted_prices[district] = {}
    for crop in crop_prices[district].columns:
        predicted_prices[district][crop] = []
        prev_price = crop_prices[district][crop].iloc[-1]  # Use the last available price as the starting point
        mean = crop_stats[crop]['Mean']
        volatility = crop_stats[crop]['Volatility']
        for year in prediction_years:
            price = mean + volatility * gauss(0, 1) 
            predicted_prices[district][crop].append(price)
            prev_price = price

# Save the predicted prices to CSV files (district-wise)
for district in predicted_prices:
    output_file = f"{district}_predicted_prices.csv"
    predicted_prices_df = pd.DataFrame(predicted_prices[district], index=prediction_years)
    predicted_prices_df.to_csv(output_file)

# Display the predicted prices for each district
for district in predicted_prices:
    print(f"District: {district}")
    output_file = f"{district}_predicted_prices.csv"
    predicted_prices_df = pd.read_csv(output_file, index_col=0)
    print(predicted_prices_df)
    print()

District: Bellary
            PADDY      SORGHUM        MAIZE  FINGER MILLET     CHICKPEA  \
2017  1283.913711  1473.380686  1247.626937    1346.317962  2733.339594   
2018  1071.603929  1195.090704  1044.414809    1043.824362  2104.187518   
2019  1247.472988   934.276091   833.663759     895.096672  2124.736484   
2020  1012.480063   602.269692   873.095590     705.322144  2049.952059   
2021   914.442174   676.716708   966.399696     850.273374  1773.017588   
2022   778.518915  1176.618218   896.946514     950.606151  1478.078360   
2023  1147.880451   755.402486   864.695151     965.339749  1954.443014   
2024  1036.556815   929.954076   685.602040    1080.568336  2653.663938   
2025  1079.441034   851.175081   644.224092     936.742380  3194.285603   
2026   957.194457  1043.845021   846.122818    1011.817705  3215.342585   
2027   568.053748  1424.046722   845.991402    1069.026012  3285.475335   
2028   973.049556   783.094078   905.426198     940.144624  1647.591532   
2029  1